In [5]:
%load_ext autoreload
%autoreload 2

In [8]:
from data_cleaning import *
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sb
from scipy import stats
from scipy.stats import pearsonr
from tqdm import tqdm

In [9]:
def dataset_creator(models_names: list, columns_names: list, k1: int):
    header = pd.MultiIndex.from_product([models_names, columns_names])
    df = pd.DataFrame(columns=header)
    df['KFold'] = np.arange(1, k1+1)
    df.set_index('KFold', inplace=True)
    return df

def twolevelcv(X: np.array, y: np.array, k1: int, k2: int, models: list, params: dict, rs: int):
    """Allows to compute two level crossvalidation.

    Args:
        X (np.array): Features (numeric)
        y (np.array): Class (objective variable)
        k1 (int): Nº of outer folds
        k2 (int): Nº of inner folds
        models (list): List of models for comparison
        params (dict): Dictionary including the set of parameters. In this case we only tune 1 parameter per model.
        rs (int): Random state
    Returns:
        df: Dataframe
    """
    test_error_dict = {}
    k = 0
    names = [type(m).__name__ for m in models]
    col_names = ['Param. Value', 'Error']
    df = dataset_creator(names, col_names, k1)
    kf1 = StratifiedKFold(k1, shuffle = True, random_state=rs)
    # first level split
    for train_idx1, test_idx1 in kf1.split(X, y):
        k += 1
        kf2 = StratifiedKFold(k2, shuffle = True, random_state=rs)
        print(f'Computing KFold {k}/{k1}...')
        # second level split
        for train_idx2, test_idx2 in tqdm(kf2.split(X[train_idx1, :], y[train_idx1]), total = k2):
            X_train = X[train_idx2, :]
            y_train = y[train_idx2]
            X_test = X[test_idx2, :]
            y_test = y[test_idx2]
            for name, model in zip(names, models):
                if name != 'DummyClassifier':
                    pname = list(params[name].keys())[0]
                    error_test = []
                    for p_ in params[name][pname]:
                        pdict = {pname: p_}
                        model = model.set_params(**pdict)
                        # train the model
                        model.fit(X_train, y_train)
                        # evaluate performance
                        pred2_test = model.predict(X_test)
                        error_test.append(np.sum(pred2_test != y_test)/ y_test.shape[0])
                    min_param = params[name][pname][np.argmin(error_test)]
                else:
                    model.fit(X_train, y_train)
                    pred2_test = model.predict(X_test)
                    error_test = np.sum(pred2_test != y_test)/ y_test.shape[0]
                    min_param = np.NaN
                df.loc(axis = 1)[name, 'Error'][k] = np.min(error_test)
                df.loc(axis = 1)[name, 'Param. Value'][k] = min_param
    return df, test_idx1

### **Data flow**

In [10]:
df_raw, df_Xn_raw = load_data()

In [11]:
fill_methods = ['mean', 'median', 'most_frequent']
std_methods = ['standard', 'minmax', 'maxabs', 'robust']

In [12]:
X, y = transform_data(df_raw, fill_methods[0], std_methods[0])

In [14]:
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.dummy import DummyRegressor
from sklearn.neural_network import MLPRegressor

random_state = 3
params = {}
lam = np.logspace(-6, 2, 100)
C = 1/ lam
# C = [200000000, 10000000, 0.1519911082952933, 0.2848035868435805 ]
params['LogisticRegression'] = {'C': C}
params['DummyRegressor'] = {'strategy': ['mean', 'median', 'quantile']}
params['MLPRegressor'] = {'hidden_layer_sizes': [(8, ), (16, ), (20, )]}
params['LinearRegression'] = {'fit_intercept': [True, False]}
params['Ridge'] = {'alpha': [0.1, 1.0, 10.0]}
models = [LogisticRegression(solver='saga', max_iter=1000000, random_state= random_state, tol = 0.003, n_jobs = -1),
          DummyRegressor(),
          MLPRegressor(solver='adam', activation='logistic', alpha=1e-4, random_state=random_state, max_iter=1000,
          early_stopping=True, validation_fraction=0.2, warm_start=True, verbose=False),
          LinearRegression(n_jobs=-1),
          Ridge(random_state=random_state)]
k1 = 10
k2 = 10
Table, test_set_outer = twolevelcv(df_raw,k1=k1,k2=k2,model=models,params=params,r_s=random_state)

NameError: name 'X_stdz' is not defined

In [1]:
import numpy as np

In [10]:
# Data management
import numpy as np
import pandas as pd
from sklearn.model_selection import (GridSearchCV,
                                    RandomizedSearchCV,
                                    cross_validate,
                                    KFold, 
                                    StratifiedKFold, 
                                    RepeatedKFold, 
                                    train_test_split, 
                                    ShuffleSplit)
from sklearn.multiclass import (OneVsRestClassifier,
                                OneVsOneClassifier)
from sklearn.preprocessing import LabelEncoder, label_binarize, OneHotEncoder, MinMaxScaler, Normalizer, StandardScaler
from sklearn.feature_selection import SelectKBest
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from itertools import permutations

# Metrics
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score, accuracy_score, \
    mean_absolute_error, balanced_accuracy_score, make_scorer,\
    mean_absolute_percentage_error, mean_squared_error
from sklearn.model_selection import permutation_test_score

# Display
from matplotlib import style
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay, confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
from tqdm import tqdm

#--Models--
# Clasificacion
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
#from pygam import LogisticGAM, LinearGAM

# Regression
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor, BaggingRegressor
from xgboost import XGBRegressor

# Clustering
from sklearn.cluster import KMeans, AgglomerativeClustering, Birch, SpectralClustering, MeanShift
from data_cleaning import *

In [3]:
def grid(X, Y, method="Nearest Neighbors", cv=None, searcher=GridSearchCV, random_state=None, problem="classification", scoring=None, prefix=""):
  n_samples = X.shape[0]
  n_features = X.shape[1]

  p_grid = {}
  estimator = None
  clf = None
  ##
  if method == "Nearest Neighbors":
    p_grid = {"n_neighbors":[2,3,5,7,10,12],
              "weights":["uniform","distance"]
              }
    if problem == "regression":
      estimator = KNeighborsRegressor()
    else:  
      estimator = KNeighborsClassifier()
  ##
  if method == "SVM":
    p_grid = {"kernel":['linear', 'poly', 'rbf', 'sigmoid'],
              "C":[0.1,0.2,0.4, 1.0],
              "gamma":[0.2,0.4,1.4,1.0],
              "decision_function_shape":[None,'ovo','ovr']
              }
    if problem == "classification":
      estimator = SVC(random_state=random_state)
  ##
  if method == "Gaussian Process":
    p_grid = {"kernel":[1.0 * RBF(1.0)]
              }
    if problem == "regression":
      estimator = GaussianProcessRegressor(random_state=random_state)
    else:
      estimator = GaussianProcessClassifier(random_state=random_state)
  ##
  if method == "Decision Tree":
    p_grid = {"max_depth":[2,5,10]
              }
    if problem == "regression":
      estimator = DecisionTreeRegressor(random_state=random_state)
    else:
      estimator = DecisionTreeClassifier(random_state=random_state)
  ##    
  if method == "Random Forest":
    p_grid = {"max_depth":[4,5,6,7],
              "n_estimators":[2,4,8,10],
              "max_features": [2,5,50,100,1000,2500], #[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18],
              "max_features":[0.25,0.5],
              "min_samples_split":[2]
              }
    if problem == "regression": 
      estimator = RandomForestRegressor(random_state=random_state)
    else:
      estimator = RandomForestClassifier(random_state=random_state)
      p_grid["criterion"] = ["gini"]
  ##
  if method == "Neural Net":
    p_grid = {"max_iter":[5000], 
              "hidden_layer_sizes": ((16,28),(512,128),(128,64)), 
              "early_stopping":[True]
              }
    if problem == "regression":
      estimator = MLPRegressor(random_state=random_state)
    else:
      estimator = MLPClassifier(random_state=random_state)
  ##
  if method == "AdaBoost":
    p_grid = {"n_estimators": [50,100], #[5,10,20,50],
              "learning_rate": [0.0001,0.01,1.0]
              }
    if problem == "regression":
      estimator = AdaBoostRegressor(random_state=random_state)
      p_grid["loss"] = ["linear", "square"]
    else:
      estimator = AdaBoostClassifier(random_state=random_state)
      p_grid["algorithm"] = ["SAMME", "SAMME.R"]
      #p_grid["base_estimator"] = [ExtraTreesClassifier(), SVC(kernel='rbf', gamma=1.3)]
  ##
  if method == "Naive Bayes":
    p_grid = {"var_smoothing":np.logspace(0,-9, num=100)
              }
    if problem == "classification":
      estimator = GaussianNB()

  if method == "QDA":
    if problem == "classification":
      estimator = QuadraticDiscriminantAnalysis()
  ##
  if method == "Gradient Boosting":
    p_grid = {"learning_rate":[0.0001, 0.001, 0.01],
              "n_estimators": [50,100], #[5,8,10,50],
              #"subsample": [0.5,1.0],
              "criterion": ["friedman_mse", "squared_error"],
              "max_depth": [4,8]#[3,5,7]
              }
    if problem == "regression":
      estimator = GradientBoostingRegressor(random_state=random_state)
      p_grid["loss"] = ["squared_error", "absolute_error", "quantile"]
    else:
      estimator = GradientBoostingClassifier(random_state=random_state)
      p_grid["loss"] = ["deviance", "exponential"]
  ##
  if method == "Bagging":
    p_grid = {'n_estimators': [100,125],
              'max_samples': [0.5,2,4,20], # 0.25
              'max_features': [0.5,2,5,20], # 0.25
              'bootstrap': [True, False],
              'bootstrap_features': [True, False]
              }
    if problem == "regression":
      estimator = BaggingRegressor(random_state=random_state)   
    else:
      estimator = BaggingClassifier(random_state=random_state)
      #p_grid["base_estimator"] = [GradientBoostingClassifier(), XGBClassifier(), SVC(random_state=random_state)]
      #p_grid["base_estimator"] = [XGBClassifier()]
  ##
  if method == "XGBoost":
    p_grid = {'max_depth': [None,0,1,2,10],
              'n_estimators': [39,40,41],
              'learning_rate': [0.6,0.4,0.35,0.45,0.5]
              }
    if problem == "regression":
      estimator = XGBRegressor()
      p_grid["objective"] = ['reg:squarederror']
    else:
      estimator = XGBClassifier()
  ##
  if method == "Extra Trees":
    p_grid = {
         "criterion":["gini","entropy"],
         "max_depth":[None,5,10,15],
         "min_samples_split":[2,4,8],
         "min_samples_leaf":[1,2,4],
         #"min_weight_fraction_leaf":[0.0,0.5,0.75],
         #"max_features":["auto","sqrt","log2"],
         "min_impurity_decrease":[0.0,0.2],
         #"bootstrap":[True,False],
         "oob_score":[False]
    }
    if problem == "regression":
      estimator = ExtraTreesRegressor(random_state=random_state)
    else:
      estimator = ExtraTreesClassifier(random_state=random_state)
  ##
  if method == "Logistic Regression":
    p_grid = {

    }
    if problem == "classification":
      estimator = LogisticRegression(random_state=random_state)

  p_grid_ = {}
  for key in p_grid:
    p_grid_[prefix+key] = p_grid[key]

  # CVSearch & Train
  #estimator = OneVsRestClassifier(estimator)
  if estimator is not None:
    clf = searcher(estimator=estimator,
                      param_grid=p_grid_,
                      cv=cv,
                      scoring=scoring,
                      verbose=2)
                      #n_iter=10)
    clf.fit(X, Y)
  return clf

In [11]:
def grid_run(X,Y,names,problem_type,cv,scoring):
  # Ejecutamos una búsqueda Grid para cada clasificador en la lista:
  results = {}
  for name in names:
    print(f"\n//--------------------------{name}-----------------------------------------------------------------------------------------------//")
    try:
      gs = grid(X,Y,
                method = name,
                cv=cv,
                problem = problem_type,
                scoring = scoring,
                #prefix="estimator__"
                )
    except Exception as e:
      print(f"[X] Error: el entrenamiento con {name} fallado!")
      print(e)
      gs = None
    if gs != None:
      results[name] = gs.cv_results_
    else:
      results[name] = None
  return results

In [19]:
# Técnica de Split
# cv = RepeatedKFold(n_splits=10, n_repeats=10, random_state=0)
# cv = ShuffleSplit(n_splits=10, test_size=.2, random_state=0)
cv = KFold(n_splits=10, random_state=0, shuffle=True)

scoring = "neg_root_mean_squared_error" #"neg_mean_absolute_error"  "neg_mean_absolute_percentage_error" #"roc_auc_ovo_weighted" #"accuracy" # #"f1" # "neg_mean_absolute_error" # "balanced_accuracy" # rounded_regression_scorer

names = [
  "Nearest Neighbors",
  "SVM",
  #"Gaussian Process",
  #"Decision Tree",
  "Random Forest",
  "Neural Net",
  #"AdaBoost",
  #"Naive Bayes",
  #"QDA",
  "Gradient Boosting",
  "Bagging",
  "XGBoost",
  #"Extra Trees",
  #"Logistic Regression"
  ]

In [20]:
df, df_n = load_data()
fill_method = "mean"
std_method = "standard"
X, y = transform_data(df, fill_method, std_method)
problem_type = "regression"

In [21]:
results = grid_run(X, y,names,problem_type=problem_type,cv=cv,scoring=scoring)


//--------------------------Nearest Neighbors-----------------------------------------------------------------------------------------------//
Fitting 10 folds for each of 12 candidates, totalling 120 fits
[CV] END .....................n_neighbors=2, weights=uniform; total time=   0.0s
[CV] END .....................n_neighbors=2, weights=uniform; total time=   0.0s
[CV] END .....................n_neighbors=2, weights=uniform; total time=   0.0s
[CV] END .....................n_neighbors=2, weights=uniform; total time=   0.0s
[CV] END .....................n_neighbors=2, weights=uniform; total time=   0.0s
[CV] END .....................n_neighbors=2, weights=uniform; total time=   0.0s
[CV] END .....................n_neighbors=2, weights=uniform; total time=   0.0s
[CV] END .....................n_neighbors=2, weights=uniform; total time=   0.0s
[CV] END .....................n_neighbors=2, weights=uniform; total time=   0.0s
[CV] END .....................n_neighbors=2, weights=uniform; to

c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450

[CV] END early_stopping=True, hidden_layer_sizes=(16, 28), max_iter=5000; total time=   0.0s
[CV] END early_stopping=True, hidden_layer_sizes=(16, 28), max_iter=5000; total time=   0.0s
[CV] END early_stopping=True, hidden_layer_sizes=(16, 28), max_iter=5000; total time=   0.0s


c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450

[CV] END early_stopping=True, hidden_layer_sizes=(16, 28), max_iter=5000; total time=   0.2s
[CV] END early_stopping=True, hidden_layer_sizes=(16, 28), max_iter=5000; total time=   0.0s


...

c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450

[CV] END early_stopping=True, hidden_layer_sizes=(128, 64), max_iter=5000; total time=   0.1s


c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\base.py:450


//--------------------------Gradient Boosting-----------------------------------------------------------------------------------------------//
Fitting 10 folds for each of 72 candidates, totalling 720 fits
[CV] END criterion=friedman_mse, learning_rate=0.0001, loss=squared_error, max_depth=4, n_estimators=50; total time=   0.0s
[CV] END criterion=friedman_mse, learning_rate=0.0001, loss=squared_error, max_depth=4, n_estimators=50; total time=   0.0s
[CV] END criterion=friedman_mse, learning_rate=0.0001, loss=squared_error, max_depth=4, n_estimators=50; total time=   0.0s
[CV] END criterion=friedman_mse, learning_rate=0.0001, loss=squared_error, max_depth=4, n_estimators=50; total time=   0.0s
[CV] END criterion=friedman_mse, learning_rate=0.0001, loss=squared_error, max_depth=4, n_estimators=50; total time=   0.0s
[CV] END criterion=friedman_mse, learning_rate=0.0001, loss=squared_error, max_depth=4, n_estimators=50; total time=   0.0s
[CV] END criterion=friedman_mse, learning_rate=0.

c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
150 fits failed out of a total of 750.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\xgboost\core.py", line 620, in inner_f
    return func(**kwargs)
  File "c:\Users\Diego\Anaconda3\envs\ML\lib\site-packages\xgboost\sklearn.py", line 1025, in fit
    self._Booster = train(
  File "c:\Users\Diego\Anaconda3\envs\ML\li

In [22]:
from sklearn.metrics import SCORERS

In [23]:
# Mostramos resultados para cada búsqueda:
for name in names:
  print(name)
  result = results[name]
  if result is None:
    print(f"[!] Warning: Sucedió un error o no funciona con {problem_type}")
  else:
    result = results[name]
    best_estimator = np.where(result["rank_test_score"] == 1)[0][0]
    print("Best Score: ", result['mean_test_score'][best_estimator])
    print("Best Params: ", result['params'][best_estimator])
    print("Mean Score Std: ", np.nanmean(result['std_test_score']))
  print("--")

Nearest Neighbors
Best Score:  -35.380489667292935
Best Params:  {'n_neighbors': 10, 'weights': 'distance'}
Mean Score Std:  8.907195731871509
--
SVM
[!] Warning: Sucedió un error o no funciona con regression
--
Random Forest
Best Score:  -30.354658386039244
Best Params:  {'max_depth': 6, 'max_features': 0.5, 'min_samples_split': 2, 'n_estimators': 8}
Mean Score Std:  8.410532553535814
--
Neural Net
Best Score:  -28.114251811343404
Best Params:  {'early_stopping': True, 'hidden_layer_sizes': (128, 64), 'max_iter': 5000}
Mean Score Std:  9.15932265896047
--
Gradient Boosting
Best Score:  -31.45328952513255
Best Params:  {'criterion': 'squared_error', 'learning_rate': 0.01, 'loss': 'squared_error', 'max_depth': 4, 'n_estimators': 100}
Mean Score Std:  10.299335854320718
--
Bagging
Best Score:  -32.06635226022968
Best Params:  {'bootstrap': False, 'bootstrap_features': False, 'max_features': 0.5, 'max_samples': 0.5, 'n_estimators': 125}
Mean Score Std:  8.828681729799627
--
XGBoost
Best S

In [24]:
results["XGBoost"]["split0_test_score"]

array([-20.66577283, -20.66577283, -20.66577283,          nan,
                nan,          nan, -20.7862241 , -20.78622395,
       -20.55148389, -23.47990153, -23.4539962 , -23.45358483,
       -21.7912852 , -21.7912852 , -21.7912852 , -19.33004674,
       -19.32987247, -19.32961015,          nan,          nan,
                nan, -22.54118212, -21.76649624, -22.01619306,
       -16.77422646, -16.68687747, -16.72338874, -23.0018557 ,
       -23.00174441, -23.00181222, -19.53216796, -19.53197745,
       -19.53115998,          nan,          nan,          nan,
       -22.13736476, -22.15034479, -22.2865438 , -19.47169013,
       -19.52353804, -19.48908706, -18.61672413, -18.61686389,
       -18.61697119, -20.55386598, -20.55402635, -20.554023  ,
                nan,          nan,          nan, -20.36967181,
       -20.56735144, -20.82137989, -20.9675317 , -20.9949188 ,
       -21.01181433, -19.47633036, -19.47632081, -19.47632398,
       -16.30655037, -16.30655039, -16.30655018,       

In [25]:
X

,x_ 1,x_ 2,x_ 3,x_ 4,x_ 5,x_ 6,x_ 7,x_ 8,x_ 9,x_10,...,x_91,x_92,x_93,x_94,x_95,C_ 1,C_ 2,C_ 3,C_ 4,C_ 5
0,0.143986,2.260189e-02,-2.850105e-01,-7.435489e-01,2.456093e+00,1.446173e-01,-6.147271e-01,-9.121041e-01,3.566176e-01,-9.206728e-17,...,-1.649033e+00,0.138263,9.793806e-02,8.378344e-01,1.704149e+00,2.0,1.0,1.0,3.0,4.0
1,0.516806,8.667054e-01,-2.673960e-01,8.712837e-16,-3.281814e-01,-6.263430e-01,-6.124337e-01,-1.137175e+00,-8.465451e-17,-7.773419e-02,...,-2.187538e-01,-0.912484,-1.838815e-01,-2.875376e-01,6.151883e-02,1.0,1.0,1.0,2.0,2.0
2,1.212469,-9.280076e-01,-5.290959e-02,-2.100282e-01,4.789575e-01,-1.364370e-16,-1.438641e-01,1.526557e-16,-3.645890e-01,-9.206728e-17,...,-7.706964e-01,0.651022,-8.439384e-01,-1.045923e-01,-1.052754e+00,0.0,1.0,4.0,3.0,0.0
3,-0.820630,-1.389646e+00,-1.197705e+00,-1.531863e+00,-1.787671e+00,-1.364370e-16,-1.104790e+00,-1.851163e+00,-9.946757e-01,-1.114715e+00,...,-3.171861e-01,-1.352818,-1.251456e-01,-1.296980e+00,-1.106595e+00,4.0,0.0,2.0,2.0,1.0
4,-1.658707,-1.950917e-01,-7.110901e-01,-2.541152e-01,-1.968123e-16,-1.564791e+00,-3.355622e-01,2.673742e-01,-1.106392e+00,-9.206728e-17,...,1.475805e-01,-0.726516,-1.233878e-01,4.898043e-17,7.463119e-02,1.0,0.0,4.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.369381,-2.561083e-16,8.412186e-01,-1.022040e-01,-3.236794e-01,8.246634e-01,4.054057e-01,2.648181e-01,-8.561281e-02,5.743276e-02,...,1.652425e-16,1.843120,3.940871e-01,1.071092e+00,2.198565e-01,1.0,1.0,0.0,2.0,1.0
96,1.185837,6.585023e-01,-4.071356e-16,-9.172382e-02,1.079191e+00,4.372737e-01,1.723159e-16,1.202554e+00,3.350057e-01,1.787323e+00,...,7.738873e-01,1.841760,1.254235e+00,8.664569e-01,-3.291940e-17,1.0,1.0,2.0,3.0,4.0
97,-2.502159,-1.724363e+00,-1.810019e+00,-6.697596e-01,-2.055407e+00,-9.044445e-01,-2.118585e+00,-1.763739e+00,-8.465451e-17,-2.706339e+00,...,-3.245595e+00,-1.812665,-1.312082e-16,-1.586817e+00,-7.744656e-01,0.0,0.0,2.0,0.0,4.0
98,0.202403,-1.341014e+00,-1.607346e-01,8.712837e-16,-1.968123e-16,-8.982464e-02,2.719395e-01,3.551797e-01,5.843045e-01,-9.206728e-17,...,-5.407144e-01,-0.306346,8.331026e-01,-9.655927e-01,-1.284772e+00,0.0,1.0,2.0,1.0,1.0
